In [17]:
# 10/01/2023

In [3]:
import pandas as pd
import copy

In [15]:
def combine_tuning_results(tuning_results, output_address):
    """ Combines multiple tuning result DataFrames into one DataFrame"""
    
    assert type(tuning_results) == list
    assert len(tuning_results) > 1

    columns = set(tuning_results[0].columns)
    for tuning_result in tuning_results[1:]:
        assert set(tuning_result.columns) == columns
        

    combined_tuning_results = pd.DataFrame()

    for tuning_result in tuning_results:
        combined_tuning_results = combined_tuning_results.append(tuning_result)
    
    output_address_split = output_address.split('.csv')[0]

    tuning_result.to_csv(f'{output_address_split}.csv', index = False)





class YiLong:



    def __init__(self, type):
        
        # check correct input
        assert type == 'Classification' or type == 'Regression' or 'GLM Regression'

        self.clf_type = type
        self._initialise_objects() # Initialise objects
        print(f'YiLong Initialised to analyse {self.clf_type}')



    def _initialise_objects(self):
        """ Helper to initialise objects """

        self.tuning_result = None
        self.hyperparameters = None
        self._seed = 18861201

        self.regression_extra_output_columns = ['Train r2', 'Val r2', 'Test r2', 
            'Train RMSE', 'Val RMSE', 'Test RMSE', 'Train MAPE', 'Val MAPE', 'Test MAPE', 'Time']
        self.classification_extra_output_columns = ['Train accu', 'Val accu', 'Test accu', 
            'Train balanced_accu', 'Val balanced_accu', 'Test balanced_accu', 'Train f1', 'Val f1', 'Test f1', 
            'Train precision', 'Val precision', 'Test precision', 'Train recall', 'Val recall', 'Test recall', 'Time']
        self.GLM_Regression_extra_output_columns = ['Train deviance', 'Val deviance', 'Test deviance', 'Time']

        self.discard_columns = None

        self.tuning_result = None
        self.hyperparameters = None

    

    def read_tuning_result(self, address, extra_to_discard_columns = None):
        """ Read in Tuning Result """

        if extra_to_discard_columns is not None:
            assert type(extra_to_discard_columns) == list

        self.tuning_result = pd.read_csv(address)

        print(f'Successfully read in tuning result, with {len(self.tuning_result)} columns')

        # get list of hyperparameters by taking what is not in the extra_output_columns
        if self.clf_type == 'Classification':
            self.discard_columns = copy.deepcopy(self.classification_extra_output_columns)
            
            if extra_to_discard_columns is not None:
                self.discard_columns.append(extra_to_discard_columns)


        elif self.clf_type == 'Regression':
            self.discard_columns = copy.deepcopy(self.regression_extra_output_columns)
            
            if extra_to_discard_columns is not None:
                self.discard_columns.append(extra_to_discard_columns)


        elif self.clf_type == 'GLM Regression':
            self.discard_columns = copy.deepcopy(self.GLM_Regression_extra_output_columns)
            
            if extra_to_discard_columns is not None:
                self.discard_columns.append(extra_to_discard_columns)

        self.hyperparameters = [col for col in self.tuning_result.columns if col not in self.discard_columns]



    def read_sorted_full_df(self, interested_statistic = None, ascending = False):
        """ View dataframe sorted in reverse in terms of validation score """
        
        assert type(ascending) == bool

        if self.tuning_result is None:
            print('Please run read_tuning_result() first')
            return
        
        if interested_statistic is not None:
            if self.clf_type == 'Regression':
                if interested_statistic not in self.regression_extra_output_columns:
                    print('Statistic not valid for a Regression Model')
                    return

            elif self.clf_type == 'Classification':
                if interested_statistic not in self.classification_extra_output_columns:
                    print('Statistic not valid for a Classification Model')
                    return
            
            elif self.clf_type == 'GLM Regression':
                if interested_statistic not in self.GLM_Regression_extra_output_columns:
                    print('Statistic not valid for a GLM Regression Model')
                    return
        
        

        if len(self.tuning_result) < 60:
            length = len(self.tuning_result)
        else:
            length = 60


        if self.clf_type =='Regression':
            if interested_statistic == None:
                interested_statistic = 'Val r2'

            sorted_tuning_results = self.tuning_result.sort_values([interested_statistic], ascending = ascending)

        elif self.clf_type =='Classification':
            if interested_statistic == None:
                interested_statistic = 'Val accu'
            
            sorted_tuning_results = self.tuning_result.sort_values([interested_statistic], ascending = ascending)
        
        elif self.clf_type =='GLM Regression':
            if interested_statistic == None:
                interested_statistic = 'Val deviance'
            
            sorted_tuning_results = self.tuning_result.sort_values([interested_statistic], ascending = ascending)


        sorted_tuning_results.index = range(len(sorted_tuning_results))
        best_hyperparameter_combination = {hyperparameter:sorted_tuning_results.iloc[0][hyperparameter] for hyperparameter in self.hyperparameters}
        print('Best hyperameter combination:', best_hyperparameter_combination, '\n')

        print(f'Highest {length}')
        display(sorted_tuning_results.head(length))
        print(f'Lowest {length}')
        display(sorted_tuning_results.tail(length))


        return best_hyperparameter_combination 



    def read_mean_val_scores(self):
        """ View the means of evaluation metrics for combinations containing each individual value of a hyperparameter – for each hyperparameter """

        for col in self.hyperparameters: # for each hyperparameter

            print(col)

            hyperparameter_values = list(set(self.tuning_result[col]))
            hyperparameter_values.sort()
            
            # create this temporary dataframe
            validation_score_df = pd.DataFrame()
            for value in hyperparameter_values: # for each value in the hyperparameter
                tmp_df = self.tuning_result[self.tuning_result[col] == value] # select df with only those parameter values

                # get means
                if self.clf_type == 'Classification':
                    tmp_df_mean = tmp_df[self.classification_extra_output_columns[:-1]].mean().T
                elif self.clf_type == 'Regression':
                    tmp_df_mean = tmp_df[self.regression_extra_output_columns[:-1]].mean().T
                elif self.clf_type == 'GLM Regression':
                    tmp_df_mean = tmp_df[self.GLM_Regression_extra_output_columns[:-1]].mean().T

                # get number of observations in this group
                tmp_df_mean['n'] = len(tmp_df)

                # append to this temporary dataframe
                validation_score_df[f'{value}'] = tmp_df_mean

            display(validation_score_df)
    


    def read_grouped_scores(self):
        """ View all evaluation metrics for combinations grouped by containing each individual value of a hyperparameter – for each hyperparameter 
        If any of the individual values of a hyperparameter exceeds 60, then sample down to 60 without replacement """

        for col in self.hyperparameters: # for each hyperparameter

            print(col)

            hyperparameter_values = list(set(self.tuning_result[col]))
            hyperparameter_values.sort()
            
            # create this temporary dataframe
            for value in hyperparameter_values: # for each value in the hyperparameter
                tmp_df = self.tuning_result[self.tuning_result[col] == value] # select df with only those parameter values

                if len(tmp_df) < 60:
                    length = len(tmp_df)
                else:
                    length = 60

                display(tmp_df.sample(length, replace=False, random_state=self._seed))

In [2]:
yilong = YiLong('Regression')

YiLong Initialised to analyse Regression


In [4]:
yilong.read_tuning_result('../models/tuning/N_gbr_3.csv')

Successfully read in tuning result, with 2084 columns


In [5]:
yilong.read_sorted_full_df()

Best hyperameter combination: {'learning_rate': 0.01, 'n_estimators': 200.0, 'subsample': 0.5, 'max_features': 0.5, 'ccp_alpha': 0.0, 'max_depth': 5.0, 'random_state': 19260817.0} 

Highest 60


,learning_rate,n_estimators,subsample,max_features,ccp_alpha,max_depth,random_state,Train r2,Val r2,Test r2,Train RMSE,Val RMSE,Test RMSE,Train MAPE,Val MAPE,Test MAPE,Time
0,0.01,200.0,0.50,0.50,0.0000,5.0,19260817.0,0.2221,0.1286,0.1196,0.1312,0.1389,0.1397,8.403188e+13,8.933264e+13,8.892022e+13,36.07
1,0.01,200.0,0.75,0.50,0.0000,5.0,19260817.0,0.2300,0.1269,0.1171,0.1306,0.1390,0.1399,8.366106e+13,8.922785e+13,8.970578e+13,53.05
2,0.01,200.0,0.25,0.50,0.0001,10.0,19260817.0,0.2724,0.1254,0.1117,0.1269,0.1391,0.1403,8.142384e+13,8.955844e+13,9.066635e+13,35.92
3,0.01,200.0,0.50,0.75,0.0000,5.0,19260817.0,0.2281,0.1250,0.1151,0.1307,0.1392,0.1400,8.370109e+13,8.917348e+13,8.945200e+13,55.55
4,0.01,200.0,0.75,0.75,0.0000,5.0,19260817.0,0.2360,0.1243,0.1184,0.1300,0.1392,0.1398,8.319864e+13,8.926584e+13,8.938846e+13,80.77
5,0.01,150.0,0.50,0.50,0.0000,5.0,19260817.0,0.1970,0.1227,0.1152,0.1333,0.1393,0.1400,8.615189e+13,9.030063e+13,8.996476e+13,28.17
6,0.01,200.0,0.75,0.25,0.0000,5.0,19260817.0,0.2135,0.1224,0.1162,0.1319,0.1394,0.1399,8.482928e+13,8.990584e+13,8.973214e+13,25.42
7,0.01,200.0,0.25,0.25,0.0000,5.0,19260817.0,0.1927,0.1220,0.1133,0.1337,0.1394,0.1402,8.626851e+13,9.000877e+13,9.071753e+13,11.28
8,0.10,150.0,0.50,0.50,0.0001,5.0,19260817.0,0.1506,0.1217,0.1097,0.1371,0.1394,0.1405,8.707315e+13,8.840674e+13,8.881219e+13,27.55
9,0.10,50.0,0.75,0.50,0.0000,5.0,19260817.0,0.3072,0.1216,0.1151,0.1238,0.1394,0.1400,7.739576e+13,8.797220e+13,8.806469e+13,11.83


Lowest 60


,learning_rate,n_estimators,subsample,max_features,ccp_alpha,max_depth,random_state,Train r2,Val r2,Test r2,Train RMSE,Val RMSE,Test RMSE,Train MAPE,Val MAPE,Test MAPE,Time
2024,100.0,150.0,0.75,0.75,10.0,100.0,19260817.0,-inf,-inf,-inf,inf,inf,inf,inf,inf,inf,598.29
2025,100.0,150.0,0.75,0.75,100.0,50.0,19260817.0,-inf,-inf,-inf,inf,inf,inf,inf,inf,inf,297.47
2026,100.0,150.0,0.75,0.75,100.0,100.0,19260817.0,-inf,-inf,-inf,inf,inf,inf,inf,inf,inf,577.97
2027,10.0,200.0,0.25,0.50,100.0,100.0,19260817.0,-inf,-inf,-inf,inf,inf,inf,3.246834e+203,3.246841e+203,3.246757e+203,87.14
2028,100.0,150.0,0.50,0.25,100.0,100.0,19260817.0,-inf,-inf,-inf,inf,inf,inf,inf,inf,inf,117.33
2029,10.0,200.0,0.50,0.25,10.0,100.0,19260817.0,-inf,-inf,-inf,inf,inf,inf,4.378446e+203,4.378456e+203,4.378342e+203,95.09
2030,100.0,150.0,0.50,0.75,100.0,100.0,19260817.0,-inf,-inf,-inf,inf,inf,inf,inf,inf,inf,401.25
2031,10.0,200.0,0.50,0.75,10.0,100.0,19260817.0,-inf,-inf,-inf,inf,inf,inf,4.378446e+203,4.378456e+203,4.378342e+203,310.49
2032,10.0,200.0,0.50,0.75,100.0,50.0,19260817.0,-inf,-inf,-inf,inf,inf,inf,4.378446e+203,4.378456e+203,4.378342e+203,176.53
2033,10.0,200.0,0.50,0.75,100.0,100.0,19260817.0,-inf,-inf,-inf,inf,inf,inf,4.378446e+203,4.378456e+203,4.378342e+203,279.05


{'learning_rate': 0.01,
 'n_estimators': 200.0,
 'subsample': 0.5,
 'max_features': 0.5,
 'ccp_alpha': 0.0,
 'max_depth': 5.0,
 'random_state': 19260817.0}

In [6]:
yilong.read_mean_val_scores()

learning_rate


,0.0001,0.001,0.01,0.1,1.0,10.0,100.0
Train r2,8.635802e-04,1.148956e-02,5.956522e-02,1.165540e-01,7.017089e-02,-inf,-inf
Val r2,3.049383e-04,7.528460e-03,3.052674e-02,2.192661e-02,-4.359853e-01,-inf,-inf
Test r2,2.938272e-04,7.368146e-03,2.934457e-02,1.896371e-02,-4.503289e-01,-inf,-inf
Train RMSE,1.487333e-01,1.479240e-01,1.439415e-01,1.369861e-01,1.415005e-01,inf,inf
Val RMSE,1.487759e-01,1.482285e-01,1.464587e-01,1.471030e-01,1.650175e-01,inf,inf
Test RMSE,1.488753e-01,1.483334e-01,1.466433e-01,1.474222e-01,1.655739e-01,inf,inf
Train MAPE,9.964307e+13,9.906293e+13,9.568239e+13,9.130115e+13,9.737666e+13,1.670752e+203,inf
Val MAPE,9.967608e+13,9.928320e+13,9.743262e+13,9.880957e+13,1.189230e+14,1.670756e+203,inf
Test MAPE,9.967321e+13,9.927984e+13,9.744628e+13,9.901360e+13,1.197780e+14,1.670712e+203,inf
n,1.620000e+02,3.830000e+02,4.600000e+02,4.960000e+02,3.950000e+02,7.200000e+01,116.0


n_estimators


,50.0,100.0,150.0,200.0
Train r2,-2.234004e+198,4.717187e-02,-inf,-inf
Val r2,-2.276132e+198,-8.887521e-02,-inf,-inf
Test r2,-2.285600e+198,-9.574167e-02,-inf,-inf
Train RMSE,3.079357e+97,1.443075e-01,inf,inf
Val RMSE,3.105123e+97,1.518719e-01,inf,inf
Test RMSE,3.124372e+97,1.521846e-01,inf,inf
Train MAPE,4.950506e+112,9.752484e+13,inf,1.769031e+202
Val MAPE,4.977841e+112,1.039138e+14,inf,1.769035e+202
Test MAPE,5.020492e+112,1.041892e+14,inf,1.768989e+202
n,2.240000e+02,4.800000e+02,700.0,6.800000e+02


subsample


...

,0.0,0.0001,0.001,0.01,0.1,1.0,10.0,100.0
Train r2,-2.102516e+198,1.180597e-01,3.492551e-02,-1.216216e-05,-2.630843e+193,-1.224490e-05,-inf,-inf
Val r2,-2.142166e+198,7.682041e-02,3.483571e-02,-1.216216e-05,-2.631102e+193,-1.224490e-05,-inf,-inf
Test r2,-2.151077e+198,7.190459e-02,3.420306e-02,-1.216216e-05,-2.628207e+193,-1.224490e-05,-inf,-inf
Train RMSE,2.864852e+97,1.396250e-01,1.461459e-01,1.488000e-01,1.160989e+95,1.488000e-01,inf,inf
Val RMSE,2.889102e+97,1.428832e-01,1.461378e-01,1.488000e-01,1.160989e+95,1.488000e-01,inf,inf
Test RMSE,2.907219e+97,1.433214e-01,1.462745e-01,1.489000e-01,1.160989e+95,1.489000e-01,inf,inf
Train MAPE,4.512434e+112,9.109308e+13,9.716837e+13,9.928220e+13,5.110212e+110,9.927961e+13,inf,inf
Val MAPE,4.538162e+112,9.317867e+13,9.715785e+13,9.928243e+13,5.110224e+110,9.927984e+13,inf,inf
Test MAPE,4.578308e+112,9.305495e+13,9.736543e+13,9.927984e+13,5.110090e+110,9.927725e+13,inf,inf
n,2.380000e+02,1.960000e+02,1.960000e+02,2.960000e+02,3.420000e+02,2.940000e+02,243.0,279.0


max_depth


,5.0,10.0,25.0,50.0,100.0
Train r2,-1.433100e+196,1.142929e-01,-1.220339e-05,-inf,-inf
Val r2,-1.440761e+196,-2.139686e-01,-1.220339e-05,-inf,-inf
Test r2,-1.423961e+196,-2.283958e-01,-1.220339e-05,-inf,-inf
Train RMSE,1.635115e+96,1.373509e-01,1.488000e-01,inf,inf
Val RMSE,1.639048e+96,1.550637e-01,1.488000e-01,inf,inf
Test RMSE,1.633383e+96,1.557165e-01,1.489000e-01,inf,inf
Train MAPE,4.622945e+111,9.274194e+13,9.928091e+13,inf,inf
Val MAPE,4.637378e+111,1.080500e+14,9.928114e+13,inf,inf
Test MAPE,4.649358e+111,1.086222e+14,9.927855e+13,inf,inf
n,3.600000e+02,5.890000e+02,2.950000e+02,537.0,303.0


random_state


,19260817.0
Train r2,-inf
Val r2,-inf
Test r2,-inf
Train RMSE,inf
Val RMSE,inf
Test RMSE,inf
Train MAPE,inf
Val MAPE,inf
Test MAPE,inf
n,2084.0


In [7]:
yilong.read_grouped_scores()

learning_rate


,learning_rate,n_estimators,subsample,max_features,ccp_alpha,max_depth,random_state,Train r2,Val r2,Test r2,Train RMSE,Val RMSE,Test RMSE,Train MAPE,Val MAPE,Test MAPE,Time
1253,0.0001,150.0,0.50,0.25,100.0,100.0,19260817.0,-0.0000,-0.0000,-0.0000,0.1488,0.1488,0.1489,9.967833e+13,9.967856e+13,9.967596e+13,29.02
1587,0.0001,50.0,0.25,0.75,100.0,50.0,19260817.0,-0.0000,-0.0000,-0.0000,0.1488,0.1488,0.1489,9.968292e+13,9.968315e+13,9.968055e+13,14.89
1594,0.0001,100.0,0.25,0.75,100.0,50.0,19260817.0,-0.0000,-0.0000,-0.0000,0.1488,0.1488,0.1489,9.967962e+13,9.967986e+13,9.967725e+13,29.69
1585,0.0001,50.0,0.25,0.75,10.0,50.0,19260817.0,-0.0000,-0.0000,-0.0000,0.1488,0.1488,0.1489,9.968292e+13,9.968315e+13,9.968055e+13,14.89
1309,0.0001,150.0,0.50,0.75,10.0,50.0,19260817.0,-0.0000,-0.0000,-0.0000,0.1488,0.1488,0.1489,9.967833e+13,9.967856e+13,9.967596e+13,99.95
34,0.0001,50.0,0.75,0.25,0.1,5.0,19260817.0,-0.0000,-0.0000,-0.0000,0.1488,0.1488,0.1489,9.968498e+13,9.968522e+13,9.968261e+13,6.28
1441,0.0001,200.0,0.75,0.50,10.0,50.0,19260817.0,-0.0000,-0.0000,-0.0000,0.1488,0.1488,0.1489,9.968446e+13,9.968470e+13,9.968209e+13,137.07
1462,0.0001,150.0,0.75,0.75,10.0,100.0,19260817.0,-0.0000,-0.0000,-0.0000,0.1488,0.1488,0.1489,9.968329e+13,9.968352e+13,9.968091e+13,156.90
1595,0.0001,100.0,0.25,0.75,100.0,100.0,19260817.0,-0.0000,-0.0000,-0.0000,0.1488,0.1488,0.1489,9.967962e+13,9.967986e+13,9.967725e+13,29.65
1540,0.0001,100.0,0.25,0.25,100.0,100.0,19260817.0,-0.0000,-0.0000,-0.0000,0.1488,0.1488,0.1489,9.967962e+13,9.967986e+13,9.967725e+13,9.02


,learning_rate,n_estimators,subsample,max_features,ccp_alpha,max_depth,random_state,Train r2,Val r2,Test r2,Train RMSE,Val RMSE,Test RMSE,Train MAPE,Val MAPE,Test MAPE,Time
1939,0.001,150.0,0.25,0.25,0.0000,10.0,19260817.0,0.0686,0.0344,0.0320,0.1436,0.1462,0.1465,9.681510e+13,9.872290e+13,9.891250e+13,13.37
1294,0.001,200.0,0.25,0.50,10.0000,100.0,19260817.0,-0.0000,-0.0000,-0.0000,0.1488,0.1488,0.1489,9.962439e+13,9.962462e+13,9.962202e+13,38.02
2020,0.001,150.0,0.75,0.25,0.0010,5.0,19260817.0,0.0069,0.0068,0.0067,0.1483,0.1483,0.1484,9.930913e+13,9.932923e+13,9.933169e+13,19.06
1901,0.001,200.0,0.50,0.75,0.0001,5.0,19260817.0,0.0407,0.0354,0.0356,0.1457,0.1461,0.1462,9.745359e+13,9.768497e+13,9.761398e+13,54.13
936,0.001,100.0,0.75,0.50,0.0100,10.0,19260817.0,-0.0000,-0.0000,-0.0000,0.1488,0.1488,0.1489,9.965864e+13,9.965887e+13,9.965626e+13,43.75
1450,0.001,150.0,0.75,0.50,10.0000,100.0,19260817.0,-0.0000,-0.0000,-0.0000,0.1488,0.1488,0.1489,9.966256e+13,9.966280e+13,9.966019e+13,102.59
907,0.001,100.0,0.50,0.25,1.0000,25.0,19260817.0,-0.0000,-0.0000,-0.0000,0.1488,0.1488,0.1489,9.965382e+13,9.965405e+13,9.965145e+13,17.93
901,0.001,100.0,0.50,0.25,0.0100,25.0,19260817.0,-0.0000,-0.0000,-0.0000,0.1488,0.1488,0.1489,9.965382e+13,9.965405e+13,9.965145e+13,18.11
1902,0.001,200.0,0.50,0.75,0.0001,10.0,19260817.0,0.0440,0.0359,0.0362,0.1455,0.1461,0.1461,9.729019e+13,9.764633e+13,9.758961e+13,104.80
1302,0.001,200.0,0.50,0.50,10.0000,100.0,19260817.0,-0.0000,-0.0000,-0.0000,0.1488,0.1488,0.1489,9.963098e+13,9.963122e+13,9.962861e+13,84.92


,learning_rate,n_estimators,subsample,max_features,ccp_alpha,max_depth,random_state,Train r2,Val r2,Test r2,Train RMSE,Val RMSE,Test RMSE,Train MAPE,Val MAPE,Test MAPE,Time
2054,0.01,200.0,0.25,0.25,0.0010,5.0,19260817.0,0.0347,0.0347,0.0348,0.1462,0.1462,0.1462,9.751908e+13,9.746568e+13,9.759855e+13,9.56
2051,0.01,150.0,0.50,0.75,0.0010,10.0,19260817.0,0.0334,0.0327,0.0326,0.1463,0.1463,0.1464,9.735994e+13,9.734118e+13,9.747947e+13,78.30
1129,0.01,50.0,0.75,0.25,0.0001,5.0,19260817.0,0.0623,0.0601,0.0607,0.1441,0.1442,0.1443,9.603065e+13,9.623610e+13,9.579258e+13,5.71
1635,0.01,200.0,0.25,0.25,100.0000,100.0,19260817.0,-0.0000,-0.0000,-0.0000,0.1488,0.1488,0.1489,9.949917e+13,9.949940e+13,9.949680e+13,18.00
1626,0.01,150.0,0.50,0.25,100.0000,50.0,19260817.0,-0.0000,-0.0000,-0.0000,0.1488,0.1488,0.1489,9.927370e+13,9.927393e+13,9.927133e+13,29.38
1854,0.01,200.0,0.50,0.50,0.0010,10.0,19260817.0,0.0331,0.0328,0.0324,0.1463,0.1463,0.1464,9.759306e+13,9.761317e+13,9.769494e+13,72.18
321,0.01,200.0,0.25,0.50,0.1000,50.0,19260817.0,-0.0000,-0.0000,-0.0000,0.1488,0.1488,0.1489,9.949917e+13,9.949940e+13,9.949680e+13,44.82
1631,0.01,150.0,0.50,0.50,100.0000,100.0,19260817.0,-0.0000,-0.0000,-0.0000,0.1488,0.1488,0.1489,9.927370e+13,9.927393e+13,9.927133e+13,65.58
1962,0.01,150.0,0.25,0.25,0.0000,5.0,19260817.0,0.1725,0.1166,0.1096,0.1353,0.1398,0.1405,8.804379e+13,9.083662e+13,9.171429e+13,7.35
1046,0.01,100.0,0.50,0.50,0.0001,5.0,19260817.0,0.1095,0.0979,0.0957,0.1404,0.1413,0.1416,9.240104e+13,9.301960e+13,9.291138e+13,16.24


,learning_rate,n_estimators,subsample,max_features,ccp_alpha,max_depth,random_state,Train r2,Val r2,Test r2,Train RMSE,Val RMSE,Test RMSE,Train MAPE,Val MAPE,Test MAPE,Time
127,0.1,200.0,0.75,0.25,100.0000,100.0,19260817.0,-0.0000,-0.0000,-0.0000,0.1488,0.1488,0.1489,9.988985e+13,9.989008e+13,9.988747e+13,70.78
1930,0.1,150.0,0.75,0.75,0.0001,10.0,19260817.0,0.1240,0.1161,0.1166,0.1393,0.1399,0.1399,8.990146e+13,8.991139e+13,8.967801e+13,119.76
389,0.1,100.0,0.25,0.25,0.0100,25.0,19260817.0,-0.0000,-0.0000,-0.0000,0.1488,0.1488,0.1489,1.000869e+14,1.000871e+14,1.000845e+14,9.85
1923,0.1,150.0,0.50,0.75,0.0000,5.0,19260817.0,0.4618,0.0714,0.0652,0.1091,0.1434,0.1439,6.850717e+13,9.197034e+13,9.208485e+13,41.45
613,0.1,200.0,0.75,0.50,0.0100,25.0,19260817.0,-0.0000,-0.0000,-0.0000,0.1488,0.1488,0.1489,9.988985e+13,9.989008e+13,9.988747e+13,154.03
543,0.1,150.0,0.75,0.75,0.1000,10.0,19260817.0,-0.0000,-0.0000,-0.0000,0.1488,0.1488,0.1489,9.976987e+13,9.977010e+13,9.976749e+13,117.32
1932,0.1,200.0,0.50,0.75,0.0000,10.0,19260817.0,0.9330,0.0035,-0.0604,0.0385,0.1485,0.1533,3.725669e+13,1.161958e+14,1.200820e+14,108.44
10,0.1,50.0,0.25,0.75,0.1000,100.0,19260817.0,-0.0000,-0.0000,-0.0000,0.1488,0.1488,0.1489,1.005855e+14,1.005857e+14,1.005831e+14,16.09
546,0.1,150.0,0.75,0.75,1.0000,10.0,19260817.0,-0.0000,-0.0000,-0.0000,0.1488,0.1488,0.1489,9.976987e+13,9.977010e+13,9.976749e+13,115.75
521,0.1,150.0,0.50,0.75,1.0000,50.0,19260817.0,-0.0000,-0.0000,-0.0000,0.1488,0.1488,0.1489,9.890625e+13,9.890648e+13,9.890390e+13,115.06


...

,learning_rate,n_estimators,subsample,max_features,ccp_alpha,max_depth,random_state,Train r2,Val r2,Test r2,Train RMSE,Val RMSE,Test RMSE,Train MAPE,Val MAPE,Test MAPE,Time
691,1.000,100.0,0.75,0.25,1.00,25.0,19260817.0,-0.0000,-0.0000,-0.0000,0.1488,0.1488,0.1489,1.005749e+14,1.005751e+14,1.005725e+14,29.34
712,1.000,150.0,0.25,0.25,0.01,25.0,19260817.0,-0.0001,-0.0001,-0.0001,0.1488,0.1488,0.1489,9.447511e+13,9.447533e+13,9.447286e+13,12.26
934,0.001,100.0,0.75,0.25,1.00,25.0,19260817.0,-0.0000,-0.0000,-0.0000,0.1488,0.1488,0.1489,9.965864e+13,9.965887e+13,9.965626e+13,28.90
694,1.000,100.0,0.75,0.50,0.01,25.0,19260817.0,-0.0000,-0.0000,-0.0000,0.1488,0.1488,0.1489,1.005749e+14,1.005751e+14,1.005725e+14,65.28
371,0.010,200.0,0.75,0.50,0.01,25.0,19260817.0,-0.0000,-0.0000,-0.0000,0.1488,0.1488,0.1489,9.971318e+13,9.971342e+13,9.971081e+13,156.06
248,0.010,150.0,0.25,0.75,0.10,25.0,19260817.0,-0.0000,-0.0000,-0.0000,0.1488,0.1488,0.1489,9.927913e+13,9.927936e+13,9.927676e+13,48.21
350,0.010,200.0,0.50,0.50,1.00,25.0,19260817.0,-0.0000,-0.0000,-0.0000,0.1488,0.1488,0.1489,9.950029e+13,9.950052e+13,9.949792e+13,96.36
922,0.001,100.0,0.50,0.75,0.10,25.0,19260817.0,-0.0000,-0.0000,-0.0000,0.1488,0.1488,0.1489,9.965382e+13,9.965405e+13,9.965145e+13,63.48
946,0.001,100.0,0.75,0.75,0.01,25.0,19260817.0,-0.0000,-0.0000,-0.0000,0.1488,0.1488,0.1489,9.965864e+13,9.965887e+13,9.965626e+13,98.98
832,1.000,200.0,0.50,0.50,0.10,25.0,19260817.0,-0.0000,-0.0000,-0.0000,0.1488,0.1488,0.1489,9.851796e+13,9.851819e+13,9.851562e+13,80.62


,learning_rate,n_estimators,subsample,max_features,ccp_alpha,max_depth,random_state,Train r2,Val r2,Test r2,Train RMSE,Val RMSE,Test RMSE,Train MAPE,Val MAPE,Test MAPE,Time
504,0.1000,150.0,0.50,0.25,1.00,50.0,19260817.0,-0.000000e+00,-0.000000e+00,-0.000000e+00,1.488000e-01,1.488000e-01,1.489000e-01,9.890625e+13,9.890648e+13,9.890390e+13,32.26
623,0.1000,200.0,0.75,0.75,0.01,50.0,19260817.0,-0.000000e+00,-0.000000e+00,-0.000000e+00,1.488000e-01,1.488000e-01,1.489000e-01,9.988985e+13,9.989008e+13,9.988747e+13,200.73
923,0.0010,100.0,0.50,0.75,0.10,50.0,19260817.0,-0.000000e+00,-0.000000e+00,-0.000000e+00,1.488000e-01,1.488000e-01,1.489000e-01,9.965382e+13,9.965405e+13,9.965145e+13,62.79
941,0.0010,100.0,0.75,0.50,0.10,50.0,19260817.0,-0.000000e+00,-0.000000e+00,-0.000000e+00,1.488000e-01,1.488000e-01,1.489000e-01,9.965864e+13,9.965887e+13,9.965626e+13,64.97
1679,1.0000,150.0,0.25,0.25,10.00,50.0,19260817.0,-1.000000e-04,-1.000000e-04,-1.000000e-04,1.488000e-01,1.488000e-01,1.489000e-01,9.447511e+13,9.447533e+13,9.447286e+13,14.71
590,0.1000,200.0,0.50,0.50,0.10,50.0,19260817.0,-0.000000e+00,-0.000000e+00,-0.000000e+00,1.488000e-01,1.488000e-01,1.489000e-01,9.967588e+13,9.967612e+13,9.967351e+13,97.12
1689,1.0000,150.0,0.50,0.25,100.00,50.0,19260817.0,-0.000000e+00,-0.000000e+00,-0.000000e+00,1.488000e-01,1.488000e-01,1.489000e-01,9.766684e+13,9.766706e+13,9.766451e+13,32.21
1608,0.0010,100.0,0.25,0.75,10.00,50.0,19260817.0,-0.000000e+00,-0.000000e+00,-0.000000e+00,1.488000e-01,1.488000e-01,1.489000e-01,9.962736e+13,9.962759e+13,9.962499e+13,29.65
1569,0.0010,100.0,0.25,0.25,10.00,50.0,19260817.0,-0.000000e+00,-0.000000e+00,-0.000000e+00,1.488000e-01,1.488000e-01,1.489000e-01,9.962736e+13,9.962759e+13,9.962499e+13,9.03
1517,100.0000,150.0,0.75,0.75,100.00,50.0,19260817.0,-inf,-inf,-inf,inf,inf,inf,inf,inf,inf,297.47


,learning_rate,n_estimators,subsample,max_features,ccp_alpha,max_depth,random_state,Train r2,Val r2,Test r2,Train RMSE,Val RMSE,Test RMSE,Train MAPE,Val MAPE,Test MAPE,Time
93,0.0001,50.0,0.25,0.75,0.1,100.0,19260817.0,-0.000000e+00,-0.000000e+00,-0.000000e+00,1.488000e-01,1.488000e-01,1.489000e-01,9.968292e+13,9.968315e+13,9.968055e+13,16.17
1617,0.0100,150.0,0.25,0.25,10.0,100.0,19260817.0,-0.000000e+00,-0.000000e+00,-0.000000e+00,1.488000e-01,1.488000e-01,1.489000e-01,9.927913e+13,9.927936e+13,9.927676e+13,13.54
1312,0.0001,150.0,0.50,0.75,100.0,100.0,19260817.0,-0.000000e+00,-0.000000e+00,-0.000000e+00,1.488000e-01,1.488000e-01,1.489000e-01,9.967833e+13,9.967856e+13,9.967596e+13,102.85
52,0.1000,50.0,0.75,0.25,0.0,100.0,19260817.0,9.995000e-01,3.990000e-02,3.000000e-03,3.400000e-03,1.458000e-01,1.486000e-01,2.896825e+12,1.083651e+14,1.119556e+14,66.12
1306,0.0001,150.0,0.25,0.75,10.0,100.0,19260817.0,-0.000000e+00,-0.000000e+00,-0.000000e+00,1.488000e-01,1.488000e-01,1.489000e-01,9.967827e+13,9.967850e+13,9.967590e+13,43.86
1266,0.0001,200.0,0.50,0.25,10.0,100.0,19260817.0,-0.000000e+00,-0.000000e+00,-0.000000e+00,1.488000e-01,1.488000e-01,1.489000e-01,9.967963e+13,9.967986e+13,9.967726e+13,38.79
1546,0.0001,100.0,0.50,0.25,10.0,100.0,19260817.0,-0.000000e+00,-0.000000e+00,-0.000000e+00,1.488000e-01,1.488000e-01,1.489000e-01,9.968236e+13,9.968259e+13,9.967998e+13,19.31
1497,100.0000,150.0,0.75,0.50,10.0,100.0,19260817.0,-inf,-inf,-inf,inf,inf,inf,inf,inf,inf,389.06
1396,100.0000,200.0,0.50,0.50,10.0,100.0,19260817.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,283.69
14,0.0001,200.0,0.75,0.25,0.1,100.0,19260817.0,-0.000000e+00,-0.000000e+00,-0.000000e+00,1.488000e-01,1.488000e-01,1.489000e-01,9.968446e+13,9.968470e+13,9.968209e+13,71.15


random_state


,learning_rate,n_estimators,subsample,max_features,ccp_alpha,max_depth,random_state,Train r2,Val r2,Test r2,Train RMSE,Val RMSE,Test RMSE,Train MAPE,Val MAPE,Test MAPE,Time
1890,1.0000,200.0,0.75,0.50,0.0010,10.0,19260817.0,6.570000e-02,6.260000e-02,6.220000e-02,1.438000e-01,1.440000e-01,1.441000e-01,9.273303e+13,9.305748e+13,9.337757e+13,102.07
435,0.1000,100.0,0.50,0.75,0.0100,50.0,19260817.0,-0.000000e+00,-0.000000e+00,-0.000000e+00,1.488000e-01,1.488000e-01,1.489000e-01,9.902278e+13,9.902301e+13,9.902042e+13,73.65
287,0.0100,150.0,0.75,0.25,1.0000,25.0,19260817.0,-0.000000e+00,-0.000000e+00,-0.000000e+00,1.488000e-01,1.488000e-01,1.489000e-01,9.959148e+13,9.959171e+13,9.958911e+13,51.41
2081,0.1000,200.0,0.50,0.75,0.0010,10.0,19260817.0,3.820000e-02,3.760000e-02,3.740000e-02,1.459000e-01,1.459000e-01,1.460000e-01,9.739339e+13,9.738257e+13,9.743968e+13,107.08
82,0.0001,200.0,0.75,0.75,100.0000,5.0,19260817.0,-0.000000e+00,-0.000000e+00,-0.000000e+00,1.488000e-01,1.488000e-01,1.489000e-01,9.968446e+13,9.968470e+13,9.968209e+13,83.43
1476,10.0000,150.0,0.75,0.25,10.0000,50.0,19260817.0,-5.053562e+281,-5.054060e+281,-5.048497e+281,1.057635e+140,1.057635e+140,1.057635e+140,4.655289e+155,4.655299e+155,4.655178e+155,38.20
1415,100.0000,150.0,0.25,0.75,10.0000,50.0,19260817.0,-inf,-inf,-inf,inf,inf,inf,inf,inf,inf,104.36
549,0.1000,200.0,0.25,0.25,0.0100,10.0,19260817.0,-0.000000e+00,-0.000000e+00,-0.000000e+00,1.488000e-01,1.488000e-01,1.489000e-01,1.001561e+14,1.001563e+14,1.001537e+14,16.57
911,0.0010,100.0,0.50,0.50,0.0100,50.0,19260817.0,-0.000000e+00,-0.000000e+00,-0.000000e+00,1.488000e-01,1.488000e-01,1.489000e-01,9.965382e+13,9.965405e+13,9.965145e+13,39.97
1177,0.0100,50.0,0.75,0.25,0.0010,10.0,19260817.0,1.710000e-02,1.690000e-02,1.670000e-02,1.475000e-01,1.475000e-01,1.476000e-01,9.873353e+13,9.879055e+13,9.877926e+13,10.90
